## Movie Revenue Prediction 

### Objective: Your client is a movie studio and they need to be able to predict movie revenue in order to greenlight the project and assign a budget to it. 
- Most of the data is comprised of categorical variables. 
- While the budget for the movie is known in the dataset, it is often an unknown variable during the greenlighting process. 

In [1]:
%load_ext watermark
%watermark -a "Emily Schoof" -d -t -v -p numpy,pandas,matplotlib

Emily Schoof 2019-08-22 09:33:55 

CPython 3.7.3
IPython 7.4.0

numpy 1.16.2
pandas 0.24.2
matplotlib 3.0.3


## Section 1: Data Preprocessing
- Load Movie_Revenue_Predictions.csv data
- Cleaning Data and Exploration

In [2]:
# Import necessary modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [3]:
# Load the dataset
movie_df = pd.DataFrame(pd.read_csv('Movie_Revenue_Predictions.csv'))
movie_df.head(1)

,title,tagline,revenue,budget,genres,homepage,id,keywords,original_language,overview,production_companies,production_countries,release_date,runtime,spoken_languages,status
0,Avatar,Enter the World of Pandora.,2787965087,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,"In the 22nd century, a paraplegic Marine is di...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",12/10/09,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released


In [4]:
# Assess shape of data
movie_df.shape

(4803, 16)

In [5]:
# Assess dataframe
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 16 columns):
title                   4803 non-null object
tagline                 3959 non-null object
revenue                 4803 non-null int64
budget                  4803 non-null int64
genres                  4803 non-null object
homepage                1712 non-null object
id                      4803 non-null int64
keywords                4803 non-null object
original_language       4803 non-null object
overview                4800 non-null object
production_companies    4803 non-null object
production_countries    4803 non-null object
release_date            4802 non-null object
runtime                 4801 non-null float64
spoken_languages        4803 non-null object
status                  4803 non-null object
dtypes: float64(1), int64(3), object(12)
memory usage: 600.5+ KB


In [6]:
movie_df.isnull().sum()

title                      0
tagline                  844
revenue                    0
budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
overview                   3
production_companies       0
production_countries       0
release_date               1
runtime                    2
spoken_languages           0
status                     0
dtype: int64

## Plan of Action
#### 1. Numerical columns:
   - **runtime** (2 NaN)
       - since there are so few instances, using SimpleImputer to fill in NaN values with the mean will not skew the dataset.
   - **budget**
       - since budget often an unknown variable during the greenlighting process, it should not be a feature in a model to predict revenue and can be dropped. 
   - **id**
       - this column does not appear to have any tangible data, so it can be dropped. 
        
#### 2. Categorical/Object columns:
Filling in NaN categorical values in the remaining columns is a bit tricky since there is no easily-applied statistical method.
   - **title** 
       - Since the movie title makes an initial impact on a consumer's decision to see a movie, NLP processing will be applied to this column in order to assess if certain words and the overall length of the title has an impact on the resulting movie revenue.
   - **homepage** (3091 NaN)
        - 3/4 of the data is missing (3091 of the total 4803), so this column cannot be effectively utilized for this model and should be dropped.
   - **overview** (3 NaN)
        - since there are so few instances, dropping the NaN rows will not interfere with the analysis of movie revenue. 
   - **release_date** (1 NaN)
        - since there are so few instances, dropping the NaN rows will not interfere with the analysis of movie revenue. 
   - **tagline** (844 NaN)
        - while 844 is well under 1/4 of the total data, the tagline for a movie may in fact have a significant impact on movie revenue due to its marketing implications. However, since there are already columns for keywords and overview, this column will be dropped for simplicity and possibly re-addressed later.*

In [7]:
# Drop NaNs in Numerical Columns - only select rows where runtime and release_date columns are "not null"
movie_df = movie_df.dropna(subset=['runtime', 'release_date'])
len(movie_df)

4800

In [8]:
# Convert Dates to Datetime Objects
movie_df['release_date_dt'] = pd.to_datetime(movie_df['release_date'], infer_datetime_format=True)
movie_df['release_date_dt'].head(2)

0   2009-12-10
1   2007-05-19
Name: release_date_dt, dtype: datetime64[ns]

In [9]:
# Drop Unnecessary Columns and Columns with Too Many NaNs *(> 50% of entries)* to Resolve
movie_df = movie_df.drop(columns=['id', 'budget', 'tagline', 'overview', 'homepage', 'release_date'])
movie_df.head(2)

,title,revenue,genres,keywords,original_language,production_companies,production_countries,runtime,spoken_languages,status,release_date_dt
0,Avatar,2787965087,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,2009-12-10
1,Pirates of the Caribbean: At World's End,961000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,2007-05-19


In [10]:
# Test output
movie_df.isnull().sum()

title                   0
revenue                 0
genres                  0
keywords                0
original_language       0
production_companies    0
production_countries    0
runtime                 0
spoken_languages        0
status                  0
release_date_dt         0
dtype: int64

## Explore the Values within Each Column

In [11]:
# Import helpful modules
from collections import Counter

**Title**

In [12]:
print(len(movie_df.title.value_counts(ascending=False)))
print(movie_df.title.describe())
movie_df.title.value_counts(ascending=False).nlargest()

4797
count         4800
unique        4797
top       The Host
freq             2
Name: title, dtype: object


The Host              2
Out of the Blue       2
Batman                2
Cool Runnings         1
DOA: Dead or Alive    1
Name: title, dtype: int64

**Revenue**

In [13]:
print(len(movie_df.revenue.value_counts(ascending=False)))
print(movie_df.revenue.describe())
movie_df.revenue.value_counts(ascending=False).nlargest()

3297
count    4.800000e+03
mean     8.231205e+07
std      1.628950e+08
min      0.000000e+00
25%      0.000000e+00
50%      1.918199e+07
75%      9.293886e+07
max      2.787965e+09
Name: revenue, dtype: float64


0           1424
7000000        6
8000000        6
6000000        5
12000000       5
Name: revenue, dtype: int64

*There are 1424 movies within the dataset that have a reported $0 for revenue. This is most likely due to missing data. Since movie revenue is the target variable, I'm going to convert all the '0' entries to the column mean. Since. in general, movie revenue is expected to increase over time (purely due to inflation, if nothing else), the mean movie revenue within the column could skew the data. Therefore, the mean revenue will be calculated in groups of 10 years, from the beginning year of the dataset, to the end. See more in 'release_date' assessment.*

**Genres**

In [14]:
print(movie_df.genres.describe())
print(len(movie_df.genres.value_counts(ascending=False)))
most_common, num_most_common = Counter(movie_df.genres).most_common(1)[0]
print(most_common, num_most_common)
movie_df.genres.value_counts(ascending=False).nlargest()

count                              4800
unique                             1175
top       [{"id": 18, "name": "Drama"}]
freq                                369
Name: genres, dtype: object
1175
[{"id": 18, "name": "Drama"}] 369


[{"id": 18, "name": "Drama"}]                                       369
[{"id": 35, "name": "Comedy"}]                                      282
[{"id": 18, "name": "Drama"}, {"id": 10749, "name": "Romance"}]     164
[{"id": 35, "name": "Comedy"}, {"id": 10749, "name": "Romance"}]    144
[{"id": 35, "name": "Comedy"}, {"id": 18, "name": "Drama"}]         142
Name: genres, dtype: int64

**Keywords**

In [15]:
print(len(movie_df.keywords.value_counts(ascending=False)))
print(movie_df.keywords.describe())
most_common, num_most_common = Counter(movie_df.keywords).most_common(1)[0]
print(most_common, num_most_common)
movie_df.keywords.value_counts(ascending=False).nlargest()

4220
count     4800
unique    4220
top         []
freq       411
Name: keywords, dtype: object
[] 411


[]                                                  411
[{"id": 10183, "name": "independent film"}]          55
[{"id": 187056, "name": "woman director"}]           42
[{"id": 179431, "name": "duringcreditsstinger"}]     15
[{"id": 6075, "name": "sport"}]                      13
Name: keywords, dtype: int64

*This column appears to have nested information that needs to be extracted, cleaned, then added back to the movie_df dataset. In addition, the most common keyword is '[ ]', which is empty and holds no value.*

**Original Language**

In [16]:
print(len(movie_df.original_language.value_counts(ascending=False)))
print(movie_df.original_language.describe())
movie_df.original_language.value_counts(ascending=False).nlargest()

37
count     4800
unique      37
top         en
freq      4503
Name: original_language, dtype: object


en    4503
fr      70
es      32
de      27
zh      27
Name: original_language, dtype: int64

**Production Companies**

In [17]:
print(len(movie_df.production_companies.value_counts(ascending=False)))
print(movie_df.production_companies.describe())
most_common, num_most_common = Counter(movie_df.production_companies).most_common(1)[0]
print(most_common, num_most_common)
movie_df.production_companies.value_counts(ascending=False).nlargest()

3695
count     4800
unique    3695
top         []
freq       350
Name: production_companies, dtype: object
[] 350


[]                                            350
[{"name": "Paramount Pictures", "id": 4}]      58
[{"name": "Universal Pictures", "id": 33}]     45
[{"name": "New Line Cinema", "id": 12}]        38
[{"name": "Columbia Pictures", "id": 5}]       37
Name: production_companies, dtype: int64

*This column appears to have nested information that needs to be extracted, cleaned, then added back to the movie_df dataset. In addition, the most common production_companies is '[ ]', which is empty and holds no value.*

In [18]:
no_company = movie_df[movie_df['production_companies'] == '[]']
print(len(no_company))
no_company.head(2)

350


,title,revenue,genres,keywords,original_language,production_companies,production_countries,runtime,spoken_languages,status,release_date_dt
1011,The Tooth Fairy,0,"[{""id"": 27, ""name"": ""Horror""}]","[{""id"": 10292, ""name"": ""gore""}, {""id"": 12339, ...",de,[],[],0.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,2006-08-08
1360,There Be Dragons,0,"[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 5509, ""name"": ""spanish civil war""}, {""...",en,[],[],112.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,2011-03-25


**Production Countries**

In [19]:
print(len(movie_df.production_countries.value_counts(ascending=False)))
print(movie_df.production_countries.describe())
most_common, num_most_common = Counter(movie_df.production_countries).most_common(1)[0]
print(most_common, num_most_common)
movie_df.production_countries.value_counts(ascending=False).nlargest()

469
count                                                  4800
unique                                                  469
top       [{"iso_3166_1": "US", "name": "United States o...
freq                                                   2977
Name: production_countries, dtype: object
[{"iso_3166_1": "US", "name": "United States of America"}] 2977


[{"iso_3166_1": "US", "name": "United States of America"}]                                                    2977
[{"iso_3166_1": "GB", "name": "United Kingdom"}, {"iso_3166_1": "US", "name": "United States of America"}]     181
[]                                                                                                             173
[{"iso_3166_1": "GB", "name": "United Kingdom"}]                                                               130
[{"iso_3166_1": "DE", "name": "Germany"}, {"iso_3166_1": "US", "name": "United States of America"}]            119
Name: production_countries, dtype: int64

*This column appears to have nested information that needs to be extracted, cleaned, then added back to the movie_df dataset. In addition, the incidence of 'US' (on it's own) as the production country appears to be the most common, making up around 50% of the dataset (2977 out of 4799). In addition, there are 172 entries of '[]' that appear to hold no information.*

**Runtime**

In [20]:
print(len(movie_df.runtime.value_counts(ascending=False)))
print(movie_df.runtime.describe())
movie_df.runtime.value_counts(ascending=False).nlargest()

156
count    4800.000000
mean      106.898125
std        22.561593
min         0.000000
25%        94.000000
50%       103.000000
75%       118.000000
max       338.000000
Name: runtime, dtype: float64


90.0     163
100.0    149
98.0     140
97.0     133
95.0     123
Name: runtime, dtype: int64

*There appear to be movies with a '0.0' runtime, which doesn't make sense, since this would imply that the movie was 0 minutes long.*

In [21]:
# Assess movie runtime by date
runtime_by_date = movie_df[['runtime', 'release_date_dt']]

# Select all non-zero values of runtime
runtime_by_date = runtime_by_date[runtime_by_date.runtime != 0.0]
runtime_by_date.head(2)

,runtime,release_date_dt
0,162.0,2009-12-10
1,169.0,2007-05-19


**Spoken Languages**

In [22]:
print(len(movie_df.spoken_languages.value_counts(ascending=False)))
print(movie_df.spoken_languages.describe())
most_common, num_most_common = Counter(movie_df.spoken_languages).most_common(1)[0]
print(most_common, num_most_common)
movie_df.spoken_languages.value_counts(ascending=False).nlargest()

544
count                                         4800
unique                                         544
top       [{"iso_639_1": "en", "name": "English"}]
freq                                          3171
Name: spoken_languages, dtype: object
[{"iso_639_1": "en", "name": "English"}] 3171


[{"iso_639_1": "en", "name": "English"}]                                                  3171
[{"iso_639_1": "en", "name": "English"}, {"iso_639_1": "es", "name": "Espa\u00f1ol"}]      127
[{"iso_639_1": "en", "name": "English"}, {"iso_639_1": "fr", "name": "Fran\u00e7ais"}]     114
[]                                                                                          84
[{"iso_639_1": "es", "name": "Espa\u00f1ol"}, {"iso_639_1": "en", "name": "English"}]       54
Name: spoken_languages, dtype: int64

*This column appears to have nested information that needs to be extracted, cleaned, then added back to the movie_df dataset. In addition, the incidence of 'English' as the spoken language appears to be the most common, making up around 75% of the dataset (3170 out of 4799). There are also 84 entries with a '[]'.*

In [23]:
no_language = movie_df[movie_df['spoken_languages'] == '[]']
print(len(no_language))
no_language.head(2)

84


,title,revenue,genres,keywords,original_language,production_companies,production_countries,runtime,spoken_languages,status,release_date_dt
492,Top Cat Begins,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 16, ""nam...","[{""id"": 209714, ""name"": ""3d""}]",es,"[{""name"": ""Anima Estudios"", ""id"": 9965}, {""nam...","[{""iso_3166_1"": ""IN"", ""name"": ""India""}, {""iso_...",89.0,[],Released,2015-10-30
1169,42,95020213,"[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 1480, ""name"": ""baseball""}, {""id"": 5565...",en,"[{""name"": ""Warner Bros."", ""id"": 6194}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",128.0,[],Released,2013-04-12


**Status**

In [24]:
print(len(movie_df.status.value_counts(ascending=False)))
print(movie_df.status.describe())
print(movie_df.status.value_counts(ascending=False))
most_common, num_most_common = Counter(movie_df.status).most_common(1)[0]
most_common, num_most_common

3
count         4800
unique           3
top       Released
freq          4792
Name: status, dtype: object
Released           4792
Rumored               5
Post Production       3
Name: status, dtype: int64


('Released', 4792)

*Since this dataset is a movie revenue prediction model, including movies that are rumored or post-production don't make sense. Thus, these entries should be removed.*

In [25]:
# Select only released movies
movie_df = movie_df[movie_df.status == 'Released']

# Drop status column
movie_df = movie_df.drop(columns='status')
len(movie_df)

4792

**Release Date**

In [26]:
print(len(movie_df.release_date_dt.value_counts(ascending=False)))
print(movie_df.release_date_dt.describe()) 
most_common, num_most_common = Counter(movie_df.release_date_dt).most_common(1)[0]
least_common, num_least_common = Counter(movie_df.release_date_dt).most_common()[-1]
print(most_common, num_most_common, least_common, num_least_common)
movie_df.release_date_dt.value_counts(ascending=False).nlargest()

3277
count                    4792
unique                   3277
top       2006-01-01 00:00:00
freq                        9
first     1969-01-01 00:00:00
last      2068-12-21 00:00:00
Name: release_date_dt, dtype: object
2006-01-01 00:00:00 9 2012-05-03 00:00:00 1


2006-01-01    9
2002-01-01    8
2004-09-03    7
1999-10-22    7
2014-12-25    7
Name: release_date_dt, dtype: int64

*The column dates range from the years 1969 to the future year of 2068. This is interesting, since movie revenue cannot be recorded for future dates, but it can be predicted (as is the goal of this analysis). Before moving forward, the movies with dates greater than 2020 should be dropped. Additionally, as mentioned above, this column will be used to group the data into 10 year increments to replace $0s with the mean movie revenue.*

In [27]:
movie_df = movie_df.loc[movie_df.release_date_dt < '2020-01-01 00:00:00']
print(len(movie_df.release_date_dt.value_counts(ascending=False)))
print(movie_df.release_date_dt.describe())

3146
count                    4660
unique                   3146
top       2006-01-01 00:00:00
freq                        9
first     1969-01-01 00:00:00
last      2017-02-03 00:00:00
Name: release_date_dt, dtype: object


*Calculate Mean Revenue in Dataset within 10-year Intervals*

In [28]:
# Define conditions
c_1 = ('1969-01-01' <= movie_df.release_date_dt) & (movie_df.release_date_dt < '1979-01-01 00:00:00')
c_2 = ('1979-01-01' <= movie_df.release_date_dt) & (movie_df.release_date_dt < '1989-01-01 00:00:00')
c_3 = ('1989-01-01' <= movie_df.release_date_dt) & (movie_df.release_date_dt < '1999-01-01 00:00:00')
c_4 = ('1999-01-01' <= movie_df.release_date_dt) & (movie_df.release_date_dt < '2009-01-01 00:00:00')
c_5 = ('2009-01-01' <= movie_df.release_date_dt) & (movie_df.release_date_dt <= '2017-02-03 00:00:00')
c_list = [c_1, c_2, c_3, c_4, c_5]

In [29]:
def calculate_mean(con_list, df, column):
    """ Calculate Mean Movie Revenue """
    means_list = []
    for condition in con_list:
        mean = round(movie_df.loc[condition, 'revenue'].mean(),2)
        means_list.append(mean)
    return means_list

In [30]:
# Calculate means based on time conditions
revenue_means = calculate_mean(c_list, movie_df, 'revenue')
revenue_means

[56973700.18, 58268128.98, 83967698.79, 74079368.23, 101277756.82]

### Splice dataset into intervals

In [31]:
movie_df = movie_df.sort_values(by='release_date_dt')
movie_df.head(2)

,title,revenue,genres,keywords,original_language,production_companies,production_countries,runtime,spoken_languages,release_date_dt
2224,Sweet Charity,20000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""id"": 978, ""name"": ""broken engagement""}, {""i...",en,"[{""name"": ""Universal Pictures"", ""id"": 33}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",149.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",1969-01-01
2959,Machine Gun McCain,0,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 28, ""name...","[{""id"": 11578, ""name"": ""mobster""}]",it,"[{""name"": ""Euro International Film (EIA)"", ""id...","[{""iso_3166_1"": ""IT"", ""name"": ""Italy""}]",116.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",1969-04-01


**c_1**

In [32]:
# make df splice
c_1_movie_df = movie_df.loc[c_1]
len(c_1_movie_df.loc[(c_1_movie_df.revenue == 0), 'revenue'])

24

In [33]:
# Replace 0 with mean
c_1_movie_df.loc[(c_1_movie_df.revenue == 0.0), 'revenue'] = revenue_means[0]
len(c_1_movie_df.loc[(c_1_movie_df.revenue == 0.0), 'revenue'])

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


0

**c_2 with revenue_means[1]**

In [34]:
# make df splice
c_2_movie_df = movie_df.loc[c_2]
len(c_2_movie_df.loc[(c_2_movie_df.revenue == 0.0), 'revenue'])

63

In [35]:
# Replace 0 with mean
c_2_movie_df.loc[(c_2_movie_df.revenue == 0.0), 'revenue'] = revenue_means[1]
len(c_2_movie_df.loc[(c_2_movie_df.revenue == 0.0), 'revenue'])

0

**c_3 0 with revenue_means[2]**

In [36]:
# make df splice
c_3_movie_df = movie_df.loc[c_3]
len(c_3_movie_df.loc[(c_3_movie_df.revenue == 0.0), 'revenue'])

167

In [37]:
# Replace 0 with mean
c_3_movie_df.loc[(c_3_movie_df.revenue == 0.0), 'revenue'] = revenue_means[2]
len(c_3_movie_df.loc[(c_3_movie_df.revenue == 0.0), 'revenue'])

0

**c_4 0 with revenue_means[3]**

In [38]:
# make df splice
c_4_movie_df = movie_df.loc[c_4]
len(c_4_movie_df.loc[(c_4_movie_df.revenue == 0.0), 'revenue'])

622

In [39]:
# Replace 0 with mean
c_4_movie_df.loc[(c_4_movie_df.revenue == 0.0), 'revenue'] = revenue_means[3]
len(c_4_movie_df.loc[(c_4_movie_df.revenue == 0.0), 'revenue'])

0

**c_5 0 with revenue_means[4]**

In [40]:
# make df splice
c_5_movie_df = movie_df.loc[c_5]
len(c_5_movie_df.loc[(c_5_movie_df.revenue == 0.0), 'revenue'])

522

In [41]:
# Replace 0 with mean
c_5_movie_df.loc[(c_5_movie_df.revenue == 0.0), 'revenue'] = revenue_means[4]
len(c_5_movie_df.loc[(c_5_movie_df.revenue == 0.0), 'revenue'])

0

**Merge df splits together into movie_df_modified**

In [42]:
movie_df_modified = pd.concat([c_1_movie_df, c_2_movie_df, c_3_movie_df, c_4_movie_df, c_5_movie_df], ignore_index=True)
len(movie_df_modified)

4660

In [43]:
movie_df_modified.head()

,title,revenue,genres,keywords,original_language,production_companies,production_countries,runtime,spoken_languages,release_date_dt
0,Sweet Charity,20000000.00,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""id"": 978, ""name"": ""broken engagement""}, {""i...",en,"[{""name"": ""Universal Pictures"", ""id"": 33}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",149.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",1969-01-01
1,Machine Gun McCain,56973700.18,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 28, ""name...","[{""id"": 11578, ""name"": ""mobster""}]",it,"[{""name"": ""Euro International Film (EIA)"", ""id...","[{""iso_3166_1"": ""IT"", ""name"": ""Italy""}]",116.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",1969-04-01
2,Midnight Cowboy,44785053.00,"[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 1879, ""name"": ""shower""}]",en,"[{""name"": ""United Artists"", ""id"": 60}, {""name""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",113.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",1969-05-25
3,The Wild Bunch,638641.00,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 37, ""...","[{""id"": 240, ""name"": ""underdog""}, {""id"": 642, ...",en,"[{""name"": ""Warner Brothers/Seven Arts"", ""id"": ...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",145.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",1969-06-17
4,Mississippi Mermaid,56973700.18,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name...","[{""id"": 911, ""name"": ""exotic island""}, {""id"": ...",fr,"[{""name"": ""Les Films du Carrosse"", ""id"": 53}, ...","[{""iso_3166_1"": ""FR"", ""name"": ""France""}, {""iso...",123.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""}]",1969-06-18


In [44]:
# Verify output
print(movie_df_modified.revenue.value_counts(ascending=False).nlargest())
print(' ')
print(movie_df_modified.title.value_counts(ascending=False).nlargest())

7.407937e+07    622
1.012778e+08    522
8.396770e+07    167
5.826813e+07     63
5.697370e+07     24
Name: revenue, dtype: int64
 
Out of the Blue             2
The Host                    2
Cool Runnings               1
The Theory of Everything    1
My Bloody Valentine         1
Name: title, dtype: int64


This looks much better. It appears data collection for this dataset must have stopped in early 2017, since no movies released in 2018 and 2019 are present in the dataset despite the maximum year elligibility being 2019.

### Drop row with the following 3 conditions:
    1. movie_df_modified.keyword == []
    2. movie_df_modified.production_companies == []
    3. movie_df_modified.runtime == 0.0
If all 3 of these conditions are met, then the resulting impact on a model built to predict a movie's revenue will be weakened; thus, these rows should be removed entirely.

In [45]:
# Drop rows with sub-optimal conditions:

# empty keyword, production company, and runtime of 0
movie_df_modified = movie_df_modified.drop(
     movie_df_modified[(movie_df_modified.runtime == 0.0) & 
              (movie_df_modified.keywords == "[]") & 
              (movie_df_modified.production_companies == "[]")].index)

# Drop rows where 
len(movie_df_modified)

4641

In [46]:
# Sort Values by revenue
movie_df_modified = movie_df_modified.sort_values(by='revenue', ascending=False)

#Reset Index
movie_df_modified = movie_df_modified.reset_index()
movie_df_modified.head(2)

,index,title,revenue,genres,keywords,original_language,production_companies,production_countries,runtime,spoken_languages,release_date_dt
0,3208,Avatar,2.787965e+09,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",2009-12-10
1,853,Titanic,1.845034e+09,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10749, ""n...","[{""id"": 2580, ""name"": ""shipwreck""}, {""id"": 298...",en,"[{""name"": ""Paramount Pictures"", ""id"": 4}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",194.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",1997-11-18


In [47]:
# Drop index
movie_df_modified = movie_df_modified.drop(columns='index')
movie_df_modified.head(2)

,title,revenue,genres,keywords,original_language,production_companies,production_countries,runtime,spoken_languages,release_date_dt
0,Avatar,2.787965e+09,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",2009-12-10
1,Titanic,1.845034e+09,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10749, ""n...","[{""id"": 2580, ""name"": ""shipwreck""}, {""id"": 298...",en,"[{""name"": ""Paramount Pictures"", ""id"": 4}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",194.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",1997-11-18


In [48]:
# Store dataframe globally
movie_df = movie_df_modified.copy()
%store movie_df

Stored 'movie_df' (DataFrame)
